In [100]:
import pyomo.environ as pyo
from pyomo.opt import SolverFactory
from pyomo.contrib.solver.solvers import gurobi_direct
from pyomo.contrib.solver.solvers import ipopt
from pyomo.environ import *

**Power loading problem**

In [51]:
import pandas as pd

In [53]:
gen_data = pd.DataFrame({'id':[0, 1, 2, 3, 4], 
                         'cost': [0.1, 0.05, 0.3, 0.4, 0.01],
                         'gen': [20, 10, 40, 50, 5]})
load_data = pd.DataFrame({'id': [0, 1, 2],
                          'load_demand': [50, 20, 30]})
load_data

,id,load_demand
0,0,50
1,1,20
2,2,30


In [54]:
Ng = len(gen_data)
Ng

5

In [ ]:
model = pyo.ConcreteModel()
model.pg = pyo.Var(range(Ng), bounds = (0, None))
# model.pg.pprint()
pg = model.pg
gen_sum = sum(pg[g] for g in gen_data.index) # variables
load_sum = sum(load_data.load_demand)

In [85]:
# constraints (in expression: optimization variables before the constant)
model.balance = pyo.Constraint(expr= gen_sum == load_sum)
model.cond = pyo.Constraint(expr = pg[3]+pg[0] >= load_data.load_demand[0])
model.limits = pyo.ConstraintList()
for g in gen_data.id:
    model.limits.add(expr = pg[g] <= gen_data.gen[g])
# Objective function
model.obj = pyo.Objective(expr=sum([pg[g]*gen_data.cost[g] for g in gen_data.index]))

'pyomo.core.base.constraint.ScalarConstraint'>) on block unknown with a new
Component (type=<class
'pyomo.core.base.constraint.AbstractScalarConstraint'>). This is usually
indicative of a modelling error. To avoid this warning, use
block.del_component() and block.add_component().
'pyomo.core.base.constraint.ScalarConstraint'>) on block unknown with a new
Component (type=<class
'pyomo.core.base.constraint.AbstractScalarConstraint'>). This is usually
indicative of a modelling error. To avoid this warning, use
block.del_component() and block.add_component().
'pyomo.core.base.constraint.ConstraintList'>) on block unknown with a new
Component (type=<class 'pyomo.core.base.constraint.ConstraintList'>). This is
usually indicative of a modelling error. To avoid this warning, use
block.del_component() and block.add_component().


In [86]:
model.pprint()

1 Var Declarations
    pg : Size=5, Index={0, 1, 2, 3, 4}
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :  None : False :  True :  Reals
          1 :     0 :  None :  None : False :  True :  Reals
          2 :     0 :  None :  None : False :  True :  Reals
          3 :     0 :  None :  None : False :  True :  Reals
          4 :     0 :  None :  None : False :  True :  Reals

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : 0.1*pg[0] + 0.05*pg[1] + 0.3*pg[2] + 0.4*pg[3] + 0.01*pg[4]

3 Constraint Declarations
    balance : Size=1, Index=None, Active=True
        Key  : Lower : Body                                  : Upper : Active
        None : 100.0 : pg[0] + pg[1] + pg[2] + pg[3] + pg[4] : 100.0 :   True
    cond : Size=1, Index=None, Active=True
        Key  : Lower : Body          : Upper : Active
        None :  50.0 : pg[3] + pg[0] :  +

In [87]:
# Solve using Gurobi direct
solver = SolverFactory('gurobi_direct')
results = solver.solve(model, tee=True)

Set parameter OutputFlag to value 1
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 11th Gen Intel(R) Core(TM) i5-1135G7 @ 2.40GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 7 rows, 5 columns and 12 nonzeros
Model fingerprint: 0x4197304e
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-02, 4e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+00, 1e+02]
Presolve removed 7 rows and 5 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.5050000e+01   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  2.505000000e+01


In [89]:
[pyo.value(pg[g]) for g in gen_data.index]

[20.0, 10.0, 35.0, 30.0, 5.0]